In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import random
import torch
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
SEED = 42
reset_seeds(SEED)

In [2]:
train = pd.read_csv('train_gpt.csv')
train.head()

,id,category,질문_1,질문_2,질문_3,질문_4,답변_1,답변_2,답변_3,답변_4,답변_5,답변_6,답변_7,답변_8,답변_9,답변_10
0,TRAIN_000,건축구조,면진장치가 뭐야?,면진장치에 사용되는 주요 기술은 무엇인가요?,면진장치란 무엇인가요?,면진장치에서 사용되는 핵심 기술은 어떤 것이 있나요?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을...",면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...,면진장치는 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 감소시키...,면진장치는 건물 지반에서 발생하는 진동 에너지를 흡수하여 건물의 안전을 유지하고 진...,면진장치는 건물에 도달하는 지반진을 흡수하여 건물의 진동을 감쇠시키는 장치입니다. ...,면진장치는 건물의 지반으로부터 전달되는 진동 세기를 낮추어서 건물 내부의 진동을 최...,"면진장치는 건물에 전달되는 지반 진동을 최소화하기 위해 설치되는 장치로, 지반으로부..."
1,TRAIN_001,건축구조,내진설계의 종류 좀 알려줘,내진설계에는 어떤 종류가 있는지 자세히 알려주실 수 있나요?,어떤 종류의 내진설계가 있나요?,어떤 종류의 내진설계가 있는지 자세하게 알려주세요.,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계에는 내진구조, 제진구조, 면진구조가 있습니다. 내진구조는 건물 구조물이 지...","내진설계에는 주로 내진구조, 제진구조, 면진구조의 세 가지 종류가 있습니다. 이들은...","내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조...","내진 설계에는 다양한 종류가 있지만, 대표적으로 내진구조, 제진구조, 면진구조가 있...","내진 설계에는 내진구조, 제진구조, 면진구조가 포함됩니다.","건물 구조물을 지진의 영향에서 보호하기 위해, 내진설계에는 내진구조, 제진구조, 면...","주로 내진구조, 제진구조 및 면진구조와 같은 세 가지 종류의 내진설계가 사용됩니다....","내진설계는 건물을 지진 영향으로부터 보호하기 위해 사용되는 주요 구조 방법으로, 건...","내진 설계에는 다양한 형태가 있지만, 주로 내진구조, 제진구조, 면진구조로 분류됩니..."
2,TRAIN_002,건축구조,철골구조의 장점이 뭐야?,철골구조의 장점을 알려줘?,철골구조의 이점은 무엇인가요?,철골구조의 장점에 대해 알려드릴까요?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점은 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건...,철골구조의 장점은 건물의 외벽에 하중이 적게 걸리기 때문에 고층 건물의 건축이 용이...,"철골구조의 장점은 건물의 외벽이 하중이 걸리지 않아 공간 활용이 용이하고, 고층 건...",철골구조의 장점은 건물의 외벽에 하중이 크게 걸리지 않아 고층 건물을 건축할 수 있...,"고층 건물을 짓는 데 유리한 점 중 하나는, 철골구조로 인해 외벽에 부담이 적게 가...",철골구조는 고층 건물을 가능하게 하는 이유는 건물 외벽에 하중이 크게 걸리지 않기 ...,철골구조는 건물 외벽에 하중이 분산되기 때문에 고층 건물을 쉽게 건설할 수 있는 장...,철골구조의 이점은 건물의 외벽이 하중을 지지하지 않아 공간을 효율적으로 활용할 수 ...,철골구조의 장점은 하중이 외벽에 크게 걸리지 않아 고층 건물을 지을 수 있는 점입니...
3,TRAIN_003,건축구조,철골철근 콘크리트 구조가 뭐야?,철골철근 콘크리트 구조의 장점과 단점에는 무엇이 있을까요?,철골철근 콘크리트 구조는 무엇인가요?,철골철근 콘크리트 구조의 장점과 단점은 무엇인가요?,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ...","철골철근콘크리트 구조는 건축물을 지탱하는 주요 구조물인 철골과 철근, 그리고 콘크리...",철골철근 콘크리트 구조는 건축물을 지탱하기 위한 구조물에서 일반적으로 사용되는 방식...,"철골철근콘크리트 구조는 철골과 철근, 그리고 콘크리트를 함께 사용하여 만들어지는 건...","철골철근 콘크리트 구조는 강철 골조와 강철 철근, 그리고 콘크리트를 함께 사용하여 ...","철근철골콘크리트는 철골과 철근, 콘크리트를 함께 사용하여 건축 구조를 형성하는 방식...","철골철근콘크리트 구조는 건축물을 지탱하는 중요한 구조로, 철골과 철근, 그리고 콘크...",철골철근 콘크리트 구조는 주로 건축물을 견고하게 지탱하기 위해 사용되는 방식입니다....,"철골철근콘크리트 구조는 건축에 사용되는 구조로, 철골과 철근 그리고 콘크리트를 함께...","철골철근 콘크리트 구조란 건축물을 구성하는데 강철 골조와 강철 철근, 그리고 콘크리..."
4,TRAIN_004,건축구조,철골구조는 어떤 방식이 있어?,철골구조의 다양한 방식이 무엇인가요?,어떤 방식으로 철골구조가 구성되는지 알려줄래?,철골 구조에는 어떤 다양한 방법이 존재할까요?,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 일반철골구조와 경량철골구조가 있습니다. 일반철골구조는 주로 대형 건물이나...,철골구조는 주로 일반철골구조와 경량철골구조로 나뉘어집니다. 이들은 건축 시스템에 따...,철골구조는 주로 일반철골구조와 경량철골구조로 구분됩니다. 이외에도 최근에는 고층 건...,철골구조는 일반철골구조와 경량철골구조 두 가지 방식이 주로 사용됩니다. 일반철골구조...,일반철골구조와 경량철골구조는 두 종류의 철 골조 시스템입니다.,일반철골구조와 경량철골구조는 건물의 크기와 용도에 따라 선택되어 사용됩니다. 일반철...,"일반철골구조와 경량철골구조는 주로 사용되는 철골구조로, 건축 시스템에 따라 선적식,...",철골구조는 주로 두 가지로 나눌 수 있습니다. 일반적으로 사용되는 일반철골구조와 경...,철골구조는 주로 일반철골구조와 경량철골구조 라는 두 가지 형태로 사용됩니다. 일반철...


In [3]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import pairwise_distances

bert_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

def evaluate_similarity(question, answer, alpha=0.95): # cosine 가중치 조절
    input_embedding = bert_model.encode(question)
    generated_embedding = bert_model.encode(answer)

    # 코사인 유사도
    cosine_sim = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0]

    # 자카드 유사도
    input_tokens = set(question.split())
    generated_tokens = set(answer.split())
    jaccard_sim = len(input_tokens.intersection(generated_tokens)) / len(input_tokens.union(generated_tokens))

    # 가중 평균 내보기
    weighted_sim = alpha*cosine_sim + (1 - alpha)*jaccard_sim

    return weighted_sim

#### 간단한 전처리 진행 - 질문

In [4]:
# gpt 질문에서 큰따옴표 삭제
train['질문_3'] = train['질문_3'].str.replace('"', '').str.replace("'", '').str.strip()
train['질문_4'] = train['질문_4'].str.replace('"', '').str.replace("'", '').str.strip()

In [5]:
# gpt 질문에서 질문이 아닌 답변 삭제
lst_1 = train.loc[((train['질문_3'].str.endswith('다.')) | (train['질문_3'].str.endswith('요.'))) , ['질문_1', '질문_3']]
lst_1 = lst_1.loc[(~(lst_1['질문_3'].str.endswith('주세요.'))) & (~(lst_1['질문_3'].str.endswith('싶습니다.'))) & (~(lst_1['질문_3'].str.endswith('궁금합니다.')))]

lst_2 = train.loc[((train['질문_4'].str.endswith('다.')) | (train['질문_4'].str.endswith('요.'))) , ['질문_2', '질문_4']]
lst_2 = lst_2.loc[~(lst_2['질문_4'].str.endswith('주세요.')) & (~(lst_2['질문_4'].str.endswith('싶습니다.'))) & (~(lst_2['질문_4'].str.endswith('궁금합니다.')))]

lst_1.shape, lst_2.shape

((69, 2), (46, 2))

In [7]:
# lst_1['index'] = lst_1.index
# lst_2['index'] = lst_2.index

# lst_1.reset_index(drop=True, inplace=True)
# lst_2.reset_index(drop=True, inplace=True)

In [8]:
# lst_1.to_csv('data/질문_1.csv', index=False, encoding='utf-8-sig')
# lst_2.to_csv('data/질문_2.csv', index=False, encoding='utf-8-sig')

In [6]:
lst_1 = pd.read_csv('질문_1_copy.csv')
lst_2 = pd.read_csv('질문_2_copy.csv')

In [7]:
for _, row in lst_1.iterrows():
    train['질문_3'][row['index']] = row['질문_1_copy']

for _, row in lst_2.iterrows():
    train['질문_4'][row['index']] = row['질문_2_copy']

In [8]:
# 67, 139, 313, 319 장단점은 수기로 수정

# 장점, 단점이 서로 다른 질문 옳은 질문으로 맞추기
idx_lst = []
for idx, tmp in train.iterrows():
    if tmp['질문_1'].find('장점') != -1:
        if tmp['질문_2'].find('단점') != -1:
            print(idx)
            print(tmp['질문_1'])
            print(tmp['질문_2'])
            print(tmp['질문_3'])
            print(tmp['질문_4'])
            print()
            idx_lst.append(idx)

195
노출콘크리트의 장점이 뭐야?
노출콘크리트의 단점은 무엇인가요?
노출콘크리트의 장점은 무엇인가요?
노출 콘크리트의 단점은 무엇인지 알려주세요.

242
알루미늄징크의 장점은 뭐야?
알루미늄징크의 단점은 무엇인가요?
알루미늄징크의 장점에는 어떤 것들이 있을까요?
알루미늄징크의 단점은 무엇이 있을까요?

257
써모사이딩의 장점은 뭐야?
써모사이딩의 단점은 무엇인가요?
써모사이딩의 이점은 무엇인가요?
써모사이딩의 단점은 무엇인지 알려줄 수 있나요?



In [9]:
for i in idx_lst:
    train.loc[i, '질문_2'] = train.loc[i, '질문_2'].replace('단점', '장점')
    train.loc[i, '질문_4'] = train.loc[i, '질문_2'].replace('단점', '장점')

In [10]:
# 장점, 단점이 서로 다른 질문 옳은 질문으로 맞추기

idx_lst = []
for idx, tmp in train.iterrows():
    if tmp['질문_1'].find('단점') != -1:
        if tmp['질문_2'].find('장점') != -1:
            print(idx)
            print(tmp['질문_1'])
            print(tmp['질문_2'])
            print(tmp['질문_3'])
            print(tmp['질문_4'])
            print()
            idx_lst.append(idx)

10
철골구조의 단점이 뭐야?
철골구조의 장점을 알려줘
철골구조의 주요 단점은 무엇인가요?
철골구조의 장점을 설명해줄래?

139
탄화목사이딩의 단점이 뭐야?
탄화목사이딩의 장점은 무엇인가요?
탄화목사이딩의 단점은 무엇인가요?
탄화목사이딩은 어떤 장점을 가지고 있을까요?

145
비드법 보온판 1종의 단점이 뭐야?
비드법 보온판 1종의 장점은 무엇인가요?
비드법 보온판 1종의 약점은 무엇인가요?
비드법 보온판 1종의 장점은 무엇인지 알려주세요.

156
점토기와의 단점이 뭐야?
점토기와의 장점은 무엇인가요?
점토기의 단점은 무엇인가요?
점토기의 장점은 무엇인가요?

159
실크벽지의 단점이 뭐야?
실크벽지의 장점은 무엇인가요?
실크벽지의 단점은 무엇인가요?
실크벽지의 장점은 무엇인지 알려줄래?

167
오리지널징크의 단점이 뭐야?
오리지널징크의 장점은 무엇인가요?
오리지널징크의 한 가지 단점은 무엇인가요?
오리지널징크의 강점은 무엇입니까?



In [13]:
for i in idx_lst:
    train.loc[i, '질문_2'] = train.loc[i, '질문_2'].replace('장점', '단점')
    train.loc[i, '질문_4'] = train.loc[i, '질문_2'].replace('장점', '단점')

In [14]:
idx_lst = []
question_lst = []
score_lst = []
question_1 = []
question_2 = []
answer = []
for idx, tmp in tqdm(train.iterrows(), total=train.shape[0]):
    for i in range(2, 5):
        if len(tmp[f'질문_{i}']) < 1:
            continue
        score = evaluate_similarity(tmp['질문_1'], tmp[f'질문_{i}'])
        if score < 0.6:
            idx_lst.append(idx)
            question_lst.append(f'질문_{i}')
            score_lst.append(score)
            question_1.append(tmp['질문_1'])
            question_2.append(f'질문_{i} : ' + tmp[f'질문_{i}'])
            answer.append(tmp[f'답변_2'])

sim_que_df = pd.DataFrame({'index':idx_lst,
                       'question':question_lst,
                       'score':score_lst,
                       '질문_1':question_1,
                       '비교':question_2,
                       '답변':answer})
sim_que_df

  0%|          | 0/644 [00:00<?, ?it/s]

,index,question,score,질문_1,비교,답변
0,42,질문_4,0.341280,사무실의 실내공기 시료채취 및 측정지점이 어떻게 돼?,질문_4 : 어떤 장비가 필요한지 알려주세요.,사무실의 실내공기 측정을 위해서는 공기질이 가장 나쁠 것으로 예상되는 2곳 이상에서...
1,72,질문_3,0.562040,준불연재료가 뭐야?,질문_3 : 준비된 불연재료는 무엇인가요?,"준불연재료는 화재에 저항이 있는 소재로, 화재에 10분 동안 견딜 수 있는 소재를 ..."
2,72,질문_4,0.372374,준불연재료가 뭐야?,질문_4 : 준비되지 않은 재료 종류를 알려주세요.,"준불연재료는 화재에 저항이 있는 소재로, 화재에 10분 동안 견딜 수 있는 소재를 ..."
3,135,질문_4,0.551117,강화마루에 대해 알려줘,질문_4 : 강화마루는 다른 바닥재료와 비교했을 때 명확하고 깔끔한 장점이 있나요?,강화마루는 원목 무늬 필름지를 입혀 하드코팅 처리한 바닥재를 말합니다. 일반적으로 ...
4,163,질문_2,0.562646,아스팔트슁글이 뭐야?,질문_2 : 아스팔트슁글은 무엇을 위해 사용되는 재료인가요?,아스팔트 슁글(Asphalt shingle)은 석유의 휘발성 물질이 증발되고 남은 ...
5,213,질문_4,0.594291,발포우레탄폼의 단점이 뭐야?,질문_4 : 발포 우레탄 폼 사용 시 주의사항을 알려주세요.,"발포우레탄폼의 단점으로는 붕레의 특성 상 매우 취약하며, 사용 중에 연소되면 유독가..."
6,233,질문_2,0.596697,내후성강판이 뭐야?,질문_2 : 내후성강판은 어떤 재료로 만들어지고 어떤 용도에 주로 사용되는지 알려주세요.,"내후성 강판은 철강재에 소량의 인, 동, 크롬, 니켈 등을 첨가하여 만든 자재로, ..."
7,233,질문_4,0.545618,내후성강판이 뭐야?,질문_4 : 제방은 내후성강판의 재료와 주로 사용되는 용도에 대해 설명해주세요.,"내후성 강판은 철강재에 소량의 인, 동, 크롬, 니켈 등을 첨가하여 만든 자재로, ..."
8,266,질문_4,0.593878,워쉬프라이머가 뭐야?,질문_4 : 워시프라이머에 대해 설명해주세요.,"워쉬프라이머는 합성수지를 기반으로 하며, 주로 뿜칠로 도장하여 방청도료의 부착성과 ..."
9,280,질문_4,0.305099,흡음재 중 판 진동형이 뭐야?,질문_4 : 조금 더 자세히 설명해 주실 수 있을까요?,"흡음재 중 판 진동형은 벽과 일정 간격을 두고 설치되는 얇고 통기성이 없는 판으로,..."


In [13]:
df = sim_que_df.reset_index(drop=True)
df.to_csv('question_1.csv', index=False, encoding='utf-8-sig')

In [15]:
del_que_df = pd.read_csv('question_1.csv')
del_que_df

,index,question,score,질문_1,비교,답변
0,42,질문_4,0.341280,사무실의 실내공기 시료채취 및 측정지점이 어떻게 돼?,질문_4 : 사무실의 실내공기 측정을 위해 어떤 장비가 필요한지 알려주세요.,사무실의 실내공기 측정을 위해서는 공기질이 가장 나쁠 것으로 예상되는 2곳 이상에서...
1,72,질문_3,0.562040,준불연재료가 뭐야?,질문_3 : 준불연재료는 무엇인가요?,"준불연재료는 화재에 저항이 있는 소재로, 화재에 10분 동안 견딜 수 있는 소재를 ..."
2,72,질문_4,0.372374,준불연재료가 뭐야?,질문_4 : 준불연재료의 종류를 알려주세요.,"준불연재료는 화재에 저항이 있는 소재로, 화재에 10분 동안 견딜 수 있는 소재를 ..."
3,135,질문_4,0.551117,강화마루에 대해 알려줘,질문_4 : 강화마루는 다른 바닥재료와 비교했을 때 명확하고 깔끔한 장점이 있나요?,강화마루는 원목 무늬 필름지를 입혀 하드코팅 처리한 바닥재를 말합니다. 일반적으로 ...
4,163,질문_2,0.562646,아스팔트슁글이 뭐야?,질문_2 : 아스팔트슁글은 무엇을 위해 사용되는 재료인가요?,아스팔트 슁글(Asphalt shingle)은 석유의 휘발성 물질이 증발되고 남은 ...
5,213,질문_4,0.594291,발포우레탄폼의 단점이 뭐야?,질문_4 : 발포우레탄폼 사용 시 주의사항을 알려주세요.,"발포우레탄폼의 단점으로는 붕레의 특성 상 매우 취약하며, 사용 중에 연소되면 유독가..."
6,233,질문_2,0.596697,내후성강판이 뭐야?,질문_2 : 내후성강판은 어떤 재료로 만들어지고 어떤 용도에 주로 사용되는지 알려주세요.,"내후성 강판은 철강재에 소량의 인, 동, 크롬, 니켈 등을 첨가하여 만든 자재로, ..."
7,233,질문_4,0.545618,내후성강판이 뭐야?,질문_4 : 내후성강판은 무엇으로 만들어지고 주로 사용되는 용도에 대해 설명해주세요.,"내후성 강판은 철강재에 소량의 인, 동, 크롬, 니켈 등을 첨가하여 만든 자재로, ..."
8,266,질문_4,0.593878,워쉬프라이머가 뭐야?,질문_4 : 워시프라이머에 대해 설명해주세요.,"워쉬프라이머는 합성수지를 기반으로 하며, 주로 뿜칠로 도장하여 방청도료의 부착성과 ..."
9,280,질문_4,0.305099,흡음재 중 판 진동형이 뭐야?,질문_4 : 판 진동형 흡음재에 조금 더 자세히 설명해 주실 수 있을까요?,"흡음재 중 판 진동형은 벽과 일정 간격을 두고 설치되는 얇고 통기성이 없는 판으로,..."


In [16]:
for _, row in del_que_df.iterrows():
    print(train.loc[row['index'], row['question']])
    train.loc[row['index'], row['question']] = row['비교'][7:]
    print(row['비교'][7:])
    print(train.loc[row['index'], row['question']])
    print()

어떤 장비가 필요한지 알려주세요.
사무실의 실내공기 측정을 위해 어떤 장비가 필요한지 알려주세요.
사무실의 실내공기 측정을 위해 어떤 장비가 필요한지 알려주세요.

준비된 불연재료는 무엇인가요?
준불연재료는 무엇인가요?
준불연재료는 무엇인가요?

준비되지 않은 재료 종류를 알려주세요.
준불연재료의 종류를 알려주세요.
준불연재료의 종류를 알려주세요.

강화마루는 다른 바닥재료와 비교했을 때 명확하고 깔끔한 장점이 있나요?
강화마루는 다른 바닥재료와 비교했을 때 명확하고 깔끔한 장점이 있나요?
강화마루는 다른 바닥재료와 비교했을 때 명확하고 깔끔한 장점이 있나요?

아스팔트슁글은 무엇을 위해 사용되는 재료인가요?
아스팔트슁글은 무엇을 위해 사용되는 재료인가요?
아스팔트슁글은 무엇을 위해 사용되는 재료인가요?

발포 우레탄 폼 사용 시 주의사항을 알려주세요.
발포우레탄폼 사용 시 주의사항을 알려주세요.
발포우레탄폼 사용 시 주의사항을 알려주세요.

내후성강판은 어떤 재료로 만들어지고 어떤 용도에 주로 사용되는지 알려주세요.
내후성강판은 어떤 재료로 만들어지고 어떤 용도에 주로 사용되는지 알려주세요.
내후성강판은 어떤 재료로 만들어지고 어떤 용도에 주로 사용되는지 알려주세요.

제방은 내후성강판의 재료와 주로 사용되는 용도에 대해 설명해주세요.
내후성강판은 무엇으로 만들어지고 주로 사용되는 용도에 대해 설명해주세요.
내후성강판은 무엇으로 만들어지고 주로 사용되는 용도에 대해 설명해주세요.

워시프라이머에 대해 설명해주세요.
워시프라이머에 대해 설명해주세요.
워시프라이머에 대해 설명해주세요.

조금 더 자세히 설명해 주실 수 있을까요?
판 진동형 흡음재에 조금 더 자세히 설명해 주실 수 있을까요?
판 진동형 흡음재에 조금 더 자세히 설명해 주실 수 있을까요?

아스팔트 슁글의 장점을 알려줄 수 있을까?
아스팔트슁글의 장점을 알려줄 수 있을까?
아스팔트슁글의 장점을 알려줄 수 있을까?

불량이란 무엇인지 말해줄래?
면 불량이란 무엇인지 말해줄래?
면 불량이란 무엇인지 말

#### 간단한 전처리 - 답변

In [17]:
# gpt 질문에서 따옴표 삭제
for i in range(6, 11):
    train[f'답변_{i}'] = train[f'답변_{i}'].str.replace('"', '').str.replace("'", '').str.strip()

In [18]:
idx_lst = []
col_lst = []
or_answer_lst = []
answer_lst = []
for idx, tmp in tqdm(train.iterrows(), total=train.shape[0]):
    for i in range(1, 6):
        if tmp[f'답변_{i}'].find('\n') == -1:
            if tmp[f'답변_{i+5}'].find('\n') != -1:
                text_1 = tmp[f'답변_{i+5}'].split('\n')[0]
                text_2 = tmp[f'답변_{i+5}'].split('\n')[1]
                score = evaluate_similarity(text_1, text_2)
                if score >= 0.9:
                    idx_lst.append(idx)
                    col_lst.append(f'답변_{i+5}')
                    or_answer_lst.append(tmp[f'답변_{i}'])
                    answer_lst.append(tmp[f'답변_{i+5}'])
                    
df = pd.DataFrame({'index':idx_lst,
                       'column':col_lst,
                       '원본답변':or_answer_lst,
                       '답변':answer_lst})
df

  0%|          | 0/644 [00:00<?, ?it/s]

,index,column,원본답변,답변
0,43,답변_9,"소화기의 주요 종류로는 ABC 소화기, 분말 소화기, 이산화탄소 소화기 등이 있습니...","ABC 소화기, 분말 소화기, 그리고 이산화탄소 소화기 등이 주로 사용되는 소화기의..."
1,78,답변_6,"데코타일은 가격이 저렴하고 시공이 간단하며 내구성, 내수성, 내마모성이 우수합니다.","데코타일은 저렴한 가격으로 구매할 수 있으며, 간편하게 시공할 수 있고, 우수한 내..."
2,99,답변_6,"화재 관련 마감재 종류는 물연재료, 준불연재료, 난연재료 등이 있습니다","화재와 관련된 마감재의 종류에는 물연재료, 준불연재료, 난연재료 등이 있습니다.\n..."
3,150,답변_6,도기질 타일의 기대수명은 50년입니다.,1. 도기질 타일의 평균 수명은 50년입니다.\n2. 도기질 타일의 예상 수명은 5...
4,173,답변_6,평지붕은 경사가 없는 지붕을 의미합니다.,평지붕은 경사가 없는 지붕이다. \n경사가 없는 지붕이 평지붕이다. \n평지붕이란 ...
5,178,답변_6,온돌마루의 기대수명은 25년입니다.,1. 온돌마루의 평균 수명은 25년이다.\n2. 온돌마루의 예상 수명은 25년이다....
6,232,답변_10,그라스울은 내부마감을 꼼꼼히 하지 않을 경우 가루가 누출될 수 있어서 관리가 조심스...,1. 그라스울을 관리할 때는 내부마감을 세심하게 처리하지 않으면 가루가 누출될 수 ...
7,239,답변_6,대리석의 기대수명은 50년입니다.,1. 대리석은 약 50년 동안 사용 가능합니다.\n2. 대리석은 약 50년 동안 서...
8,273,답변_6,유광 자기질 타일의 기대수명은 50년입니다.,- 유광 자기질 타일은 약 50년 동안 지속될 수 있는 수명을 가지고 있습니다.\n...
9,301,답변_7,천연마루의 기대수명은 일반적으로 50년입니다. 그러나 정기적인 유지보수와 적절한 관...,천연마루의 수명은 보통 50년입니다. 그러나 주기적인 유지보수와 적합한 관리를 통해...


In [19]:
import re

best_lst = []
for idx, row in df.iterrows():
    best_score = 0
    best_answer = ''
    texts = row['답변'].split('\n')
    for text in texts:
        text = text.replace('-', '').strip()
        text = re.sub('[0-9]. {1}', '', text).strip()
        score = evaluate_similarity(row['원본답변'], text)
        if score > best_score:
            best_score = score
            best_answer = text
    best_lst.append(best_answer)
    
best_lst

['ABC 소화기, 분말 소화기, 그리고 이산화탄소 소화기 등이 주로 사용되는 소화기의 주요 종류입니다. 다양한 화재 유형에 대응하기 위해 다양한 종류의 소화기가 사용되고 있습니다.',
 '데코타일은 가격이 저렴하면서도 시공이 간단하며 동시에 내구성, 내수성, 내마모성 면에서 우수한 특징을 가지고 있습니다.',
 '화재와 관련된 마감재의 종류에는 물연재료, 준불연재료, 난연재료 등이 있습니다.',
 '도기질 타일의 예상 수명은 50년입니다.',
 '평지붕이란 경사가 없는 지붕을 말한다.',
 '온돌마루의 예상 수명은 25년이다.',
 '그라스울을 관리할 때는 내부마감을 세심하게 처리하지 않으면 가루가 누출될 수 있다는 점에 주의해야 합니다. 또한, 그라스울은 유리 소재이므로 손으로 만지면 다칠 수 있는 위험이 있습니다.',
 '대리석의 수명은 대략 50년입니다.',
 '유광 자기질 타일의 예상 수명은 50년입니다.',
 '천연마루의 수명은 보통 50년입니다. 그러나 주기적인 유지보수와 적합한 관리를 통해 더 오래 사용할 수 있습니다. 이에 대해 더 자세히 설명하겠습니다.',
 '에폭시 테라죠의 기대 수명은 약 15년입니다.',
 '모자이크 타일의 예상 수명은 약 50년입니다.',
 '도배지가 화학 물질에 노출되어 오염이 발생할 수도 있습니다.',
 '마루를 갈아내거나 다시 마감하고, 또는 마루 왁스나 편리한 고무 스폰지를 사용하여 회복시켜주세요.',
 '만약 내부 도어 핸들이 느슨하게 움직인다면, 먼저 핸들을 고정하는 나사나 볼트를 조여보세요. 이 방법으로 문제가 해결되지 않는다면, 해당 부품을 교체하여 핸들을 다시 견고하게 고정해야 합니다. 이 조치를 취하면 핸들의 움직임이 줄어들고 안정적으로 작동하게 됩니다.',
 '도배풀 제거제는 도배 가게, 대형마트, 다이소 등의 매장에서 구매할 수 있습니다.',
 '수도꼭지를 분해하여 밀봉 고무링을 교체하거나 파손된 구성 요소를 교체하세요.',
 '가구 배치, 색상 팔레트 및 에리어 러그와 같은 요소를 활용하여 오픈 

In [20]:
df['수정답변'] = best_lst
for _, row in df.iterrows():
    train.loc[row['index'], row['column']] = row['수정답변']

In [21]:
train.to_csv('train_final.csv', index=False, encoding='utf-8-sig')